In [ ]:
import heapq
import pandas as pd
import math
from google.colab import files

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)
df.columns = df.columns.str.strip()

Saving A_Star.csv to A_Star (2).csv


In [ ]:
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Delhi,28.6600,77.2300,India,IN,Delhi,admin,29617000.0,16753235.0
1,Mumbai,18.9667,72.8333,India,IN,Mahārāshtra,admin,23355000.0,12478447.0
2,Kolkāta,22.5411,88.3378,India,IN,West Bengal,admin,17560000.0,4496694.0
3,Bangalore,12.9699,77.5980,India,IN,Karnātaka,admin,13707000.0,8443675.0
4,Chennai,13.0825,80.2750,India,IN,Tamil Nādu,admin,11324000.0,6727000.0


In [ ]:
expected_columns = {'city', 'lat', 'lng'}
if not expected_columns.issubset(df.columns):
    raise ValueError(f"Dataset missing required columns. Found: {df.columns}")


def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

In [ ]:
def preprocess_data(df):
    cities = df.set_index('city')[['lat', 'lng']].to_dict('index')

    graph = {}
    for city in cities:
        graph[city] = {}
        for neighbor in cities:
            if city != neighbor:
                dist = haversine(cities[city]['lat'], cities[city]['lng'],
                                 cities[neighbor]['lat'], cities[neighbor]['lng'])
                graph[city][neighbor] = dist

    return graph, cities

In [ ]:
# A* Search Algorithm
def astar(graph, cities, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_score = {city: float('inf') for city in graph}
    g_score[start] = 0

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1], g_score[goal]

        for neighbor, distance in graph[current].items():
            tentative_g_score = g_score[current] + distance
            if tentative_g_score < g_score[neighbor]:
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + haversine(
                    cities[neighbor]['lat'], cities[neighbor]['lng'],
                    cities[goal]['lat'], cities[goal]['lng']
                )
                heapq.heappush(open_list, (f_score, neighbor))
                came_from[neighbor] = current

    return None, float('inf')

graph, cities = preprocess_data(df)

start_city = 'Delhi'
goal_city = 'Jaipur'
path, cost = astar(graph, cities, start_city, goal_city)

if path:
    print("Shortest Path:", path)
    print("Total Distance:", cost, "km")
else:
    print("No path found.")

Shortest Path: ['Delhi', 'Jaipur']
Total Distance: 235.70963013737594 km
